In [ ]:
#Orchestrates preprocessing + augmentation
#%run data_augmentation and data_preprocessing

## Utilities

In [6]:
%run ../constants.py

%run data_preprocessing/domain_tagging.py 
%run data_preprocessing/feature_extraction.py 
%run data_preprocessing/text_cleaning.py
%run data_preprocessing/tokenizer.py

%run data_augmentation/paraphrasing.py
%run data_augmentation/style_transfer.py
%run data_augmentation/synonym_replacement.py

## Imports

In [1]:
import yaml

## Load config

In [3]:
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)